## PCのスペック

In [24]:
!cat /proc/cpuinfo | grep model | head -2
!cat /proc/cpuinfo | grep vendor_id   | head -1
!cat /proc/cpuinfo | grep MHz   | head -1
!cat /proc/cpuinfo | grep cache | head -1

model		: 8
model name	: AMD Ryzen 7 2700X Eight-Core Processor
vendor_id	: AuthenticAMD
cpu MHz		: 2352.228
cache size	: 512 KB


In [25]:
!cat /proc/meminfo | grep MemTotal

MemTotal:       16417760 kB


## ライブラリのインポート

In [1]:
import getfem as gf
import numpy as np

## 変換したGmsh1形式のメッシュを読み込む

In [2]:
mesh_file='foamMesh_1.msh'
degree = 2
E = 1e3
Nu = 0.3
Lambda = E*Nu / ((1+Nu)*(1-2*Nu))
Mu = E/(2*(1+Nu))

m = gf.Mesh('import', 'gmsh', mesh_file)

## 変位、応力の場と積分法を作成

In [3]:
mfu = gf.MeshFem(m,3)
mfp = gf.MeshFem(m,1)

mfu.set_fem(gf.Fem('FEM_PK(3,2)'))
mfp.set_fem(gf.Fem('FEM_PK(3,0)'))

mim = gf.MeshIm(m, gf.Integ('IM_TETRAHEDRON(5)'))

print(f'N_curves={m.nbcvs()}, N_points={m.nbpts()}, qdim={mfu.qdim()}, fem={mfu.fem()[0].char()}, DOF={mfu.nbdof()}')

N_curves=248092, N_points=50429, qdim=3, fem=FEM_PK(3,2), DOF=1094874


## 境界の名前付け

In [4]:
topfaces = m.outer_faces_in_box([0.0,-0.01,0.011], [0.025, 0.08, 0.019])
btmfaces = m.outer_faces_in_box([0.0,-0.01,-0.0061], [0.025, 0.08, -0.0059])
NEUMANN_BOUNDARY = 1
DIRICHLET_BOUNDARY = 2

m.set_region(NEUMANN_BOUNDARY,   topfaces)
m.set_region(DIRICHLET_BOUNDARY, btmfaces)

## 各項と初期条件や境界条件をセット

In [5]:
md = gf.Model('real')
md.add_fem_variable('u', mfu)
md.add_initialized_data('cmu', Mu)
md.add_initialized_data('clambda', Lambda)
md.add_isotropic_linearized_elasticity_brick(mim, 'u', 'clambda', 'cmu')
md.add_fem_variable('p', mfp)
md.add_linear_incompressibility_brick(mim, 'u', 'p')
md.add_initialized_data('VolumicData', [0,-1,0]);
md.add_source_term_brick(mim, 'u', 'VolumicData');

# Attach the tripod to the ground
md.add_Dirichlet_condition_with_multipliers(mim, 'u', mfu, 2);

In [6]:
print(f'Using memory = {md.memsize() / (1024**2):.1f} MB')

Using memory = 31.0 MB


In [7]:
md?

Type:           Model
File:           ~/Applications/getfem-5.4/build/lib/python3.6/site-packages/getfem/getfem.py
Docstring:     
GetFEM Model object

Model variables store the variables and the state data and the
description of a model. This includes the global tangent matrix, the right
hand side and the constraints. There are two kinds of models, the `real`
and the `complex` models.
Init docstring:
General constructor for Model objects

* ``MD = Model('real')``
  Build a model for real unknowns.

* ``MD = Model('complex')``
  Build a model for complex unknowns.

  


## 式を解く

In [8]:
print('running solve...')
%prun md.solve('noisy', 'max iter', 1);
U = md.variable('u');
print('solve done!')

running solve...
 solve done!


         6 function calls in 657.493 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1  657.493  657.493  657.493  657.493 {built-in method _getfem.getfem}
        1    0.000    0.000  657.493  657.493 {built-in method builtins.exec}
        1    0.000    0.000  657.493  657.493 getfem.py:2774(get)
        1    0.000    0.000  657.493  657.493 getfem.py:2915(solve)
        1    0.000    0.000  657.493  657.493 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}

## 解析結果からミーゼス応力を算出

In [9]:
mfdu=gf.MeshFem(m,1)
mfdu.set_fem(gf.Fem('FEM_PK_DISCONTINUOUS(3,1)'))
VM = md.compute_isotropic_linearized_Von_Mises_or_Tresca('u','clambda','cmu', mfdu);

# post-processing
sl=gf.Slice(('boundary',), mfu, degree)

print('Von Mises range: ', VM.min(), VM.max())

Von Mises range:  4.1703334511306625e-05 0.09285832961345553


## VTK形式で結果を出力

In [10]:
sl.export_to_vtk('mises.vtk', 'ascii', mfdu,  VM, 'Von Mises Stress', mfu, U, 'Displacement')

gf.memstats()

*** GetFEM view of the workspace:
*** Python view of the workspace:
<class 'getfem.Mesh'> class 8, id 0 : instances=1
<class 'getfem.MeshFem'> class 9, id 1 : instances=1
<class 'getfem.MeshFem'> class 9, id 2 : instances=1
<class 'getfem.MeshIm'> class 10, id 6 : instances=1
<class 'getfem.Model'> class 14, id 7 : instances=1
<class 'getfem.MeshFem'> class 9, id 8 : instances=1
<class 'getfem.Slice'> class 16, id 10 : instances=1
